In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nelgiriyewithana/most-streamed-spotify-songs-2024")

print("Path to dataset files:", path)

C:\Users\dhair\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\dhair\.cache\kagglehub\datasets\nelgiriyewithana\most-streamed-spotify-songs-2024\versions\1


In [26]:
og_spotify_data=pd.read_csv("C:/Documents/ME/DATA Analytics Journey/Spotify Analysis/Most Streamed Spotify Songs 2024.csv",encoding='latin1')

In [23]:
import pandas as pd
import numpy as np

In [24]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [31]:
og_spotify_data.head()

,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",...,684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0
1,Not Like Us,Not Like Us,Kendrick Lamar,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",...,3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1
2,i like the way you kiss me,I like the way you kiss me,Artemas,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",...,536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0
3,Flowers,Flowers - Single,Miley Cyrus,1/12/2023,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",...,"2,182",264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0
4,Houdini,Houdini,Eminem,5/31/2024,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",...,1,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1


In [93]:
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Track                   4600 non-null   object 
 1   Album Name              4600 non-null   object 
 2   Artist                  4595 non-null   object 
 3   Release Date            4600 non-null   object 
 4   ISRC                    4600 non-null   object 
 5   All Time Rank           4600 non-null   object 
 6   Track Score             4600 non-null   float64
 7   Spotify Streams         4487 non-null   object 
 8   Spotify Playlist Count  4530 non-null   object 
 9   Spotify Playlist Reach  4528 non-null   object 
 10  Spotify Popularity      3796 non-null   float64
 11  YouTube Views           4292 non-null   object 
 12  Shazam Counts           4023 non-null   object 
dtypes: float64(2), object(11)
memory usage: 467.3+ KB


In [95]:
columns_to_clean = ['Spotify Streams', 'YouTube Views', 'Shazam Counts','All Time Rank','Spotify Playlist Count','Spotify Playlist Reach']
for col in columns_to_clean:
        spotify_data[col] = spotify_data[col].str.replace(",", "", regex=True)

In [96]:
spotify_data=spotify_data.astype({
    'Release Date':'datetime64[ns]',
    'All Time Rank':'float64',
    'Spotify Streams':'float64',
    'Spotify Playlist Count':'float64',
    'Spotify Playlist Reach':'float64',
    'YouTube Views':'float64',
    'Shazam Counts':'float64'})

In [97]:
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Track                   4600 non-null   object        
 1   Album Name              4600 non-null   object        
 2   Artist                  4595 non-null   object        
 3   Release Date            4600 non-null   datetime64[ns]
 4   ISRC                    4600 non-null   object        
 5   All Time Rank           4600 non-null   float64       
 6   Track Score             4600 non-null   float64       
 7   Spotify Streams         4487 non-null   float64       
 8   Spotify Playlist Count  4530 non-null   float64       
 9   Spotify Playlist Reach  4528 non-null   float64       
 10  Spotify Popularity      3796 non-null   float64       
 11  YouTube Views           4292 non-null   float64       
 12  Shazam Counts           4023 non-null   float64 

In [98]:
spotify_data.isnull().sum()

Track                       0
Album Name                  0
Artist                      5
Release Date                0
ISRC                        0
All Time Rank               0
Track Score                 0
Spotify Streams           113
Spotify Playlist Count     70
Spotify Playlist Reach     72
Spotify Popularity        804
YouTube Views             308
Shazam Counts             577
dtype: int64

In [99]:
spotify_data=spotify_data.dropna(subset=['Artist'])

# Fill with the median
spotify_data['Spotify Streams']=spotify_data['Spotify Streams'].fillna(spotify_data['Spotify Streams'].median())
spotify_data['Spotify Playlist Count'] = spotify_data['Spotify Playlist Count'].fillna(spotify_data['Spotify Playlist Count'].median())
spotify_data['Spotify Playlist Reach'] = spotify_data['Spotify Playlist Reach'].fillna(spotify_data['Spotify Playlist Reach'].median())
spotify_data['Spotify Popularity'] = spotify_data['Spotify Popularity'].fillna(spotify_data['Spotify Popularity'].median())
# Fill with the median
spotify_data['YouTube Views'] = spotify_data['YouTube Views'].fillna(spotify_data['YouTube Views'].median())
# Fill with the median
spotify_data['Shazam Counts'] = spotify_data['Shazam Counts'].fillna(spotify_data['Shazam Counts'].median())

In [100]:
spotify_data.isnull().sum()

Track                     0
Album Name                0
Artist                    0
Release Date              0
ISRC                      0
All Time Rank             0
Track Score               0
Spotify Streams           0
Spotify Playlist Count    0
Spotify Playlist Reach    0
Spotify Popularity        0
YouTube Views             0
Shazam Counts             0
dtype: int64

In [102]:
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4595 entries, 0 to 4599
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Track                   4595 non-null   object        
 1   Album Name              4595 non-null   object        
 2   Artist                  4595 non-null   object        
 3   Release Date            4595 non-null   datetime64[ns]
 4   ISRC                    4595 non-null   object        
 5   All Time Rank           4595 non-null   float64       
 6   Track Score             4595 non-null   float64       
 7   Spotify Streams         4595 non-null   float64       
 8   Spotify Playlist Count  4595 non-null   float64       
 9   Spotify Playlist Reach  4595 non-null   float64       
 10  Spotify Popularity      4595 non-null   float64       
 11  YouTube Views           4595 non-null   float64       
 12  Shazam Counts           4595 non-null   float64      

In [103]:
CLIENT_ID="b26374f778744f2895ac3a8a498dc842"
CLIENT_SECRET="4d832a20205543539bad867a1792a40a"

client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp=spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [104]:
def get_spotify_url(track_name,artist_name):
    try:
        results=sp.search(q=f"track:{track_name} artist:{artist_name}",type="track",limit=1)
        if results['tracks']['items']:
            return results['tracks']['items'][0]['external_urls']['spotify']
        else:
            return None
    except Exception as e:
        print(f"Error while fetching url for {track_name} by {artist_name}: {e}")

#add column for url
urls=[]
for index, row in tqdm(spotify_data.iterrows(),total=len(spotify_data)):
    track=row['Track']
    artist=row['Artist']
    urls.append(get_spotify_url(track,artist))

spotify_data['Spotify URL']=urls
output_path='path'
spotify_data.to_csv(output_path, index=False, encoding='latin1')
print(f"Updated dateset saved to {output_path}")

 49%|████████████████████████████████████▉                                       | 2230/4595 [08:34<1:05:13,  1.65s/it]

Error while fetching url for Starships by Nicki Minaj: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)


100%|██████████████████████████████████████████████████████████████████████████████| 4595/4595 [17:52<00:00,  4.29it/s]

Updated dateset saved to path


In [105]:
spotify_data.head()

,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,Spotify Popularity,YouTube Views,Shazam Counts,Spotify URL
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,2024-04-26,QM24S2402528,1.0,725.4,3.904709e+08,30716.0,196631588.0,92.0,8.427475e+07,2669262.0,https://open.spotify.com/track/5AJ9hqTS2wcFQCE...
1,Not Like Us,Not Like Us,Kendrick Lamar,2024-05-04,USUG12400910,2.0,545.9,3.237039e+08,28113.0,174597137.0,92.0,1.163470e+08,1118279.0,https://open.spotify.com/track/6AI3ezQ4o3HUoP6...
2,i like the way you kiss me,I like the way you kiss me,Artemas,2024-03-19,QZJ842400387,3.0,538.4,6.013093e+08,54331.0,211607669.0,92.0,1.225991e+08,5285340.0,https://open.spotify.com/track/2GxrNKugF82CnoR...
3,Flowers,Flowers - Single,Miley Cyrus,2023-01-12,USSM12209777,4.0,444.9,2.031281e+09,269802.0,136569078.0,85.0,1.096101e+09,11822942.0,https://open.spotify.com/track/7DSAEUvxU8FajXt...
4,Houdini,Houdini,Eminem,2024-05-31,USUG12403398,5.0,423.3,1.070349e+08,7223.0,151469874.0,88.0,7.737396e+07,457017.0,https://open.spotify.com/track/1CDQzbCz4KSQxHe...


In [106]:
spotify_data.isnull().sum()

Track                       0
Album Name                  0
Artist                      0
Release Date                0
ISRC                        0
All Time Rank               0
Track Score                 0
Spotify Streams             0
Spotify Playlist Count      0
Spotify Playlist Reach      0
Spotify Popularity          0
YouTube Views               0
Shazam Counts               0
Spotify URL               585
dtype: int64

In [108]:
spotify_data=spotify_data.dropna(subset=['Spotify URL'])

In [109]:
spotify_data.isnull().sum()

Track                     0
Album Name                0
Artist                    0
Release Date              0
ISRC                      0
All Time Rank             0
Track Score               0
Spotify Streams           0
Spotify Playlist Count    0
Spotify Playlist Reach    0
Spotify Popularity        0
YouTube Views             0
Shazam Counts             0
Spotify URL               0
dtype: int64

In [110]:
spotify_data.to_csv("C:/Documents/ME/DATA Analytics Journey/Spotify Analysis/Updated_spotify_data.csv",index=False)
print("Dataset saved successfully!")

Dataset saved successfully!
